In [2]:
import flet as ft  # type: ignore
from flet.matplotlib_chart import MatplotlibChart
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from sympy import sympify, lambdify
from sympy.abc import n
from sympy.testing.pytest import ignore_warnings
import re
import nest_asyncio
nest_asyncio.apply()
matplotlib.use("agg")

In [3]:
## Parse a string expression in a math function
# @param string containing a math expression of the variable n
# @return a Python function or Exception
#
def string_to_function(expression):
    try: 
        sympy_expr = sympify(expression)  # Parse the string expression
        f = lambdify(n, sympy_expr)
    except Exception: raise SyntaxError('Error: Please enter a correct function, as 1/(n^2+1)')
    return f  # Convert to a Python function


In [4]:
## Parse a string in an array
# @param string containing a range
# @return an array for the range or an exception
#
def read_range(range_text):
    step = 1
    if len(range_text) == 0: raise AttributeError('Error: Please enter a range')
    elif len(range_text.split(",")) == 2 and re.search(r"[^-,\d]", range_text)==None: # check if the string cointains only digits, comma, or minus symbol
        nums = range_text.split(",")
        try:
            n1, n2 = int(nums[0]), int(nums[1])
        except Exception: 
            raise ValueError('Error: Invalid range. Use two integer values separated by a comma')
        if n1 < n2:
            return np.arange(n1, n2+1, step)
        else: raise AttributeError('Error: Invalid range. The first number must be lower than the second')
    else: raise AttributeError('Error: The range must be like number1,number2')


In [5]:
## compute the probabilities
# @param x, an array with the values of the axis x
# @param prob_function, a Python function
# return an array with the normalized probabilities
#
def probabilities(x, prob_function):
    with ignore_warnings(RuntimeWarning):
        probs = np.array([prob_function(x_i) for x_i in x]) 
        try:
            if np.isinf(probs).any() or np.isnan(probs).any(): raise ValueError()
        except ValueError: raise ValueError('Error: Invalid division by zero. Check the distribution or the range')
        except Exception: raise TypeError('Error: Please enter a correct function, as 1/(n^2+1)')
    almost_one_gt0 = False
    e = 10**(-10)
    for p in probs:
        if p>0: 
            almost_one_gt0 = True
        elif -e < p < e:
            pass
        else: 
            raise ValueError('Error: Invalid probability value. It must be positive. Check the distribution or the range')
    if not almost_one_gt0: raise ValueError('Error: All probability values are equale to zero. Change function or range')
    return probs / np.sum(probs)


In [6]:
## compute the entropy
# @param an array with probability values
# @return the entropy value
#
def entropy(probs):
    return np.sum( [-p_i * np.log2(p_i) for p_i in probs if p_i>0] )


In [7]:
## main part of the application, manages the GUI, using Flet library
#
def main(page: ft.Page):
    page.title = 'Exercise 1 - Part A'
    page.window.width = 750
    page.window.height = 450

    def compute(e):
        dis = distrib.value
        ran = rangev.value
        try:
            x = read_range(ran)
            fun = string_to_function(dis)
            probs = probabilities(x, fun)
            en = entropy(probs)
            entr.value = f"{en:.6f}"  # Display result with formatting

            # Clear previous chart in the chart column
            chart_column.controls.clear()
            # Create the new plot and add it to the chart column
            chart_column.controls.append(create_graph(x, probs))
           
            # Reset input
            errors.value = ""
        except Exception as ex:
            errors.value = str(ex)  # Display exception as a string

        page.update()

    def create_graph(x, y):
            fig, ax = plt.subplots()
            ax.plot(x, y, marker='o')
            ax.set(xlabel='n', ylabel='p(n)')
            chart = MatplotlibChart(fig) 
            return chart    

    # UI Setup
    page.add(ft.Text('Discrete Entropy Calculator', width=750, height=50, 
                     text_align=ft.TextAlign.CENTER, weight=ft.FontWeight.BOLD, size=20))

    distrib = ft.TextField(value="1/(n^2+1)", width=120, height=50, on_blur=compute)
    rangev = ft.TextField(value="0,10", width=120, height=50, on_blur=compute)
    entr = ft.TextField(width=120, height=50, read_only=True)
    entr.value = f"{entropy(probabilities(read_range(rangev.value), string_to_function(distrib.value))):.6f}" # initial entropy value based on starting values
    exit_button = ft.ElevatedButton("EXIT", on_click= lambda l: page.window.close())
    
    # Create the starting plot based on the initial filled values of range and distribution
    chart = create_graph(read_range(rangev.value), probabilities(read_range(rangev.value), string_to_function(distrib.value)))
    
    # Input Column    
    r1 = ft.Row(controls=[ft.Text('p(n) proportional to:'), distrib], height=50, alignment=ft.MainAxisAlignment.END)
    r2 = ft.Row(controls=[ft.Text('Range of n'), rangev], height=50, alignment=ft.MainAxisAlignment.END)
    r3 = ft.Row(controls=[ft.Text('Entropy ='), entr], height=50, alignment=ft.MainAxisAlignment.END)
    r4 = ft.Row(controls=[exit_button], alignment=ft.MainAxisAlignment.CENTER)
    input_column = ft.Column(controls=[r1, r2, r3, r4], horizontal_alignment=ft.MainAxisAlignment.START, spacing=10)
    
    # Chart Column
    chart_column = ft.Column(controls=[chart], spacing=10, expand=True)
    
    # Arrange columns with Containers to control width
    main_row = ft.Row(
        controls=[
            ft.Container(content=input_column, expand=3),
            ft.Container(content=chart_column, expand=4)
        ],
        alignment=ft.MainAxisAlignment.CENTER
    )
    
    # Add everything to the page
    page.add(main_row)
    
    errors = ft.Text()  # Error message area
    page.add(errors)
    
    page.update()

In [ ]:
# Run the app
# Works only on Windows 11 and macOS. If using Windows 10, comment this line and use the web browser option below.
ft.app(target=main,view=ft.AppView.FLET_APP)

In [ ]:
# Works on all operating systems, including Windows 10. Uncomment this line if using Windows 10 and comment the pop-up window option.
#ft.app(target=main, view=ft.AppView.WEB_BROWSER)